# CS 4501 Algorithmic Economics - Project 1

**Note:** For each of the question, please add some print or graph drawing commands to show your results in a clear way and also necessary analyses and demonstrations to help people who are not in your group understand your logics and results.


## Part 1
### Question 1
Using a Jupyter notebook import the csv file as pandas dataframe.

In [5]:
import pandas as pd
import numpy as np
import time as time
from tqdm import tqdm
from sklearn import linear_model as lm
df = pd.read_csv('./scanner_data.csv', index_col=0) # Tell pandas to ignore index column
print(df)
# Note: data format is DD/MM/YYYY and starts at 2/1/2016
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

print(  "Number of Customer_ID",    len(pd.unique(df['Customer_ID']))  )
print(  "Number of SKU  ",    len(pd.unique(df['SKU']))  )
print(max(df['Date']))
print(df)

              Date  Customer_ID  Transaction_ID SKU_Category    SKU  Quantity  \
1       02/01/2016         2547               1          X52  0EM7L       1.0   
2       02/01/2016          822               2          2ML  68BRQ       1.0   
3       02/01/2016         3686               3          0H2  CZUZX       1.0   
4       02/01/2016         3719               4          0H2  549KK       1.0   
5       02/01/2016         9200               5          0H2  K8EHH       1.0   
...            ...          ...             ...          ...    ...       ...   
131702  04/07/2016        20203           32900          IEV  FO112       3.0   
131703  04/07/2016        20203           32900          N8U  I36F2       1.0   
131704  04/07/2016        20203           32900          U5F  4X8P4       1.0   
131705  04/07/2016        20203           32900          0H2  ZVTO4       1.0   
131706  04/07/2016        20203           32900          Q4N  QM9BP       1.0   

        Sales_Amount  
1   

### Question 2
The fact that consumer does not purchase anything can be 
interpreted as that she chose an outside option. Given that the fact that she chose an outside option is not recorded in this dataset, argue how you would construct a proxi variable for the choice of an outside option. Add such a proxi variable to your dataframe. 

**Hint:** you can use information that some consumers do not appear in the data every week. 

**Please input your answer in this cell:**




Add such a proxi variable to your dataframe. 


In [14]:
import sys
from io import StringIO
from csv import writer
np.set_printoptions(threshold=1000)


df['chose_other'] = 0
df['week'] = 0
customers = df['Customer_ID'].unique()
df2 = df
week = 0
grouped = df.groupby(pd.Grouper(key='Date', freq='W'))
print(len(df), 'transactions in original data')
for key, item in tqdm(grouped):
    bought_this_week = item['Customer_ID'].unique()
    # print('For week starting in', item['Date'].iloc[0])
    chose_other = set(customers) - set(bought_this_week) # Finds everyone who didn't buy this week
    # print(type(item))
    # print(item.loc[len(item.index)-1])
    date = item['Date'].iloc[0]
    # print(date)
    # print(type(date))
    # print(len(chose_other), 'chose outside')

    num_other = len(chose_other)
    other_purchases = pd.DataFrame({'Date': [date for x in range(num_other)],
                                    'Customer_ID': [id for id in chose_other],
                                    'chose_other': [1 for x in range(num_other)],
                                    'SKU': ["0" for x in range(num_other)],
                                   'week': [week for x in range(num_other)]})
    
    df2 = pd.concat([df2, other_purchases.sample(frac=0.05)], ignore_index = True, axis=0)
    # print(df.iloc[-5:])
    
    # NOTE: this method probably works, but pd.concat() is much more efficient
    # for id in chose_other:
    #   df.loc[len(df.index)] = [date, id, None, None, None, None, None, 1, week]
    
    week += 1 
 
#Filter the data
df2 = df2.sort_values(by='SKU')
SKUIndices = {}
SKU = pd.unique(df2['SKU'])
for i in tqdm(range(0,len(SKU))):
    SKUIndices[SKU[i]] = i;
    
customers = pd.unique(df['Customer_ID'])
numCust = len(customers)
numProd = len(SKU)

print(len(df2), 'transactions after adding outside choices')

        
        

131706 transactions in original data


100%|██████████████████████████████████| 5243/5243 [00:00<00:00, 1669759.75it/s]

188757 transactions after adding outside choices


### Question 3
Given that we do not have **explicit** consumer feature vectors $\mathbf{x}^i = (x^i_1, \cdots, x_k^i)$  in the data, discuss how you would construct such feature vectors for each consumer $i$ from the given data. Add your constructed characteristics to your dataframe. 

**Hint:** you can use transaction history and argue that past shopping patterns may give a good characterization for a given consumer.

**Please input your answer in this cell:**





Add your constructed characteristics to your dataframe. 

In [42]:
featureCount = 8
custFeatureArr = np.zeros((numCust,featureCount))

for i in tqdm(range(numCust)):
    # First Organize Data & Calculate Features
    transactions = df[df.Customer_ID == customers[i]]
    custFeatureArr[i,0] = transactions.Quantity.sum()
    custFeatureArr[i,1] = transactions.Sales_Amount.sum()
    custFeatureArr[i,2] = custFeatureArr[i,1]/len(transactions.Date.unique())
    custFeatureArr[i,3] = len(transactions.Date.unique())
    custFeatureArr[i,4] = len(transactions.SKU.unique())
    custFeatureArr[i,5] = len(transactions.SKU_Category.unique())
    custFeatureArr[i,6] = len(transactions.Date.dt.isocalendar().week.unique())
    custFeatureArr[i,7] = 1

    
custFeatureArr[:,0:7] = (custFeatureArr[:,0:7] - np.mean(custFeatureArr[:,0:7],axis=0))/np.std(custFeatureArr[:,0:7],axis=0)

prices = np.zeros((numProd,1))
print(prices.shape)
prices[:,0] = df2.iloc[np.unique(df2.SKU.values,return_index=True)[1]].Sales_Amount / df2.iloc[np.unique(df2.SKU.values,return_index=True)[1]].Quantity
prices[0,0] = 0

prices[:,:] = (prices[:,:] - np.mean(prices[:,:],axis=0))/np.std(prices[:,:],axis=0)
    

100%|████████████████████████████████████| 22625/22625 [00:47<00:00, 472.52it/s]

(5243, 1)


In [135]:
prices[:,:] = (prices[:,:] - np.mean(prices[:,:],axis=0))/np.std(prices[:,:],axis=0)

### Question 4
Produce the utility parameters $\beta_{0j}, \beta_{1j},\cdots \beta_{kj}$ and $\alpha_j$ for every product $j$  by estimating a multinomial 
logit model from your constructed dataset.

In [232]:
#Hint: you can use sklearn.linear_model.LogisticRegression() to achieve an estimation
shuffledArr = np.concatenate((np.reshape(df2.Customer_ID.values,(len(df2),1)), np.reshape(df2.SKU.values,(len(df2),1))),axis=1)
np.random.shuffle(shuffledArr)

size = len(df2)
train = int(9*len(df2)/10)
test = len(df2) - train

IDs = shuffledArr[0:size,0]
cats = shuffledArr[0:size,1]


X = np.zeros((size,featureCount + 1))
y = np.zeros(size)


for i in tqdm(range(size)):
    X[i,0:featureCount] = custFeatureArr[IDs[i] - 1]
    X[i,featureCount] = prices[SKUIndices[cats[i]]]
    y[i] = SKUIndices[cats[i]]
    
lm.LogisticRegression(solver="lbfgs") 
lm.LogisticRegression(n_jobs=-1) 
lm.LogisticRegression(multi_class='multinomial')
    
tic = time.time()
model = lm.LogisticRegression().fit(X[0:train,:], y[0:train])
print(time.time() - tic)

%store model

# toc = time.time()
# scores = model.score(X[train:train+test,:],y[train:train+test])
# print(time.time() - toc)
# print(scores)

# print(model.predict(x_train).sum())
# print(y_train.sum())
  
# print(np.unique(y_train))
# print(np.where(model.predict(x_train) > 0))
# print(model.predict(x_train)[np.where(model.predict(x_train) > 0)])
# print(np.where(y_train > 0))
# print(y_train[np.where(model.predict(x_train) > 0)])

#print(scores.sum()/10)


100%|███████████████████████████████| 188757/188757 [00:00<00:00, 438249.95it/s]


KeyboardInterrupt: 

In [137]:
%store -r model
prediction = model.predict(X[train:train+test,:])
print(np.where(prediction > 0))
print(prediction[np.where(prediction > 0)])
print(np.where(y[train:train+test] > 0))
print(prediction[np.where(y[train:train+test] > 0)])
count = 0
secondCount = 0
thirdCount = 0
fourthCount = 0
fifthCount = 0
sixthCount = 0

for i in range(len(prediction)):
    if prediction[i] == y[train + i] and prediction[i] > 0:
        count = count + 1
    if prediction[i] == y[train + i] and prediction[i] == 0:
        secondCount = secondCount + 1
    if y[train + i] > 0 and prediction[i] > 0 and prediction[i] != y[train + i]:
        thirdCount = thirdCount + 1
    if y[train + i] > 0 and prediction[i] == 0:
        fourthCount = fourthCount + 1
    if prediction[i] > 0 and y[train + i]  == 0:
        fifthCount = fifthCount + 1
  
        
total = len(prediction)/100
print("Percent Correct Positive Classification: ",count/total)
print("Percent Correct Negative Classification: ",secondCount/total)
print("Percent Positive Misclassification: ",thirdCount/total)
print("Percent Incorrect Negative Classification: ",fourthCount/total)
print("Percent Incorrect Positive Classification: ",fifthCount/total)
        


(array([   11,    55,   153,   161,   170,   180,   248,   269,   373,
         379,   410,   415,   418,   509,   530,   534,   535,   554,
         560,   573,   597,   621,   743,   755,   765,   785,   793,
         799,   800,   816,   855,   860,   870,   900,   927,   963,
         970,   994,  1020,  1022,  1045,  1118,  1121,  1188,  1194,
        1204,  1207,  1229,  1258,  1355,  1364,  1374,  1391,  1419,
        1421,  1433,  1436,  1458,  1485,  1495,  1564,  1605,  1622,
        1632,  1652,  1675,  1682,  1777,  1796,  1825,  1828,  1842,
        1862,  1930,  1994,  2011,  2017,  2026,  2056,  2066,  2069,
        2071,  2083,  2099,  2144,  2154,  2161,  2172,  2228,  2240,
        2315,  2340,  2438,  2456,  2458,  2469,  2617,  2646,  2649,
        2655,  2707,  2711,  2729,  2731,  2740,  2747,  2768,  2787,
        2794,  2820,  2824,  2846,  2859,  2883,  2890,  2997,  3017,
        3027,  3064,  3065,  3066,  3096,  3110,  3114,  3149,  3156,
        3189,  3213

In [130]:
# from math import comb
# from decimal import *
# toc = time.time()
# scores = model.score(X[train:train+test,:],y[train:train+test])
# print(time.time() - toc)
# print(scores)
# print(model.coef_.shape)

# val = Decimal(comb(int(total*100),count))
# print( val * Decimal(((1/5243)**count)) * Decimal(((5242/5243)**(5243-count))))

# coef = model.coef_

print(coef.shape[0])
print(model.classes_.astype(int))
    

5141
[   0    1    2 ... 5240 5241 5242]


## Part 2
### Question 1
Construct a multi-armed bandit algorithm such that

1. It is randomly initialized at first and selects **one** product out of $j$ available products.
2. It updates  $\beta_{0j}, \beta_{1j},\cdots \beta_{kj}$ and $\alpha_j$  over  time by observing the utility $\widehat{u}_{ij}$ of each product $j$ it selected in the past and selects new products


In [230]:
#Hint: Try ridge regression on each arm separately,

numIters = 10
numCustomers = 1
alpha = np.sqrt((featureCount + 1) * 0)
reward = 0.1

classes = model.classes_.astype(int)
numClasses = classes.shape[0]
coef = model.coef_
indices = np.zeros((numCustomers,numIters))


shuffledCustomers = custFeatureArr[0:numCustomers,:]
np.random.shuffle(shuffledCustomers)
Vecs = np.zeros((numProd,featureCount + 1))
Arms = np.zeros((numProd,featureCount + 1,featureCount + 1))
thetas = np.zeros((numProd,featureCount + 1))
probs = np.zeros(numProd)
xVec = np.zeros((9,1))
XVecT = np.transpose(xVec)

for customer in tqdm(range(numCustomers)):
    
    xVec[0:8,0] = shuffledCustomers[customer,:]
    XVecT = np.transpose(xVec)
    
    for i in range(numIters):
        alpha = np.sqrt((featureCount + 1) * np.log(i+1))
        print(i)
        count = 1
        for j in classes:
            if (j == 0):
                continue
                
            xVec[8,:] = prices[j] 
            xVec = xVec * coef[count].reshape(9,1)
            count = count + 1
            xVecT = np.transpose(xVec)
                    
            if (i == 0):
                Arms[j,:,:] = np.identity(9)
                
            thetas[j,:] = np.dot(np.linalg.inv(Arms[j,:,:]),Vecs[j,:])
            probs[j] = np.squeeze(np.dot(np.transpose(thetas[j,:]),xVec)) + alpha * np.squeeze(np.sqrt(np.dot(np.dot(xVecT,Arms[j,:,:]),xVec)))
            
        index = np.argmax(probs)
        indices[customer,i] = probs[index]
        Arms[index,:,:] = Arms[index,:,:] + np.dot(xVec,xVecT)
        Vecs[index,:] = Vecs[index,:] + np.squeeze((reward * xVec))
        
        





  0%|                                                     | 0/1 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.92s/it]

9


In [231]:
print(indices)











[[ 0.         15.69503073 20.3172521  23.39804257 25.80032613 27.82077857
  29.59612567 31.20033778 32.67782649 34.05730373]]


### Question 2

 Draw 1000 random consumers from your data. For each consumer,  run your online learning algorithm for 100 steps. Note that this is a simulation process --- i.e., your algorithm itself does not know $\beta_{0j}, \beta_{1j},\cdots \beta_{kj}$ and $\alpha_j$, but can only observe the $\widehat{u}_{ij}$ for any product $j$ that the algorithm pulled (i.e., purchased).     
 For each randomly picked consumer $i$, compute the difference $\Delta_i$ between the  maximum utility $\max_j\widehat{u}_{ij}$ (i.e., consumer $i$'s  utility for her  favorite product) and the average utility that your algorithm
achieved at the 100th step. Compute the average of $\Delta_i$ over those 1000 consumers, and explain why there is such a difference.  

In [ ]:
def rewards_difference():
    
    
    return 



def simulation():
    
    
    return 




    



Explain why there is such a difference.

**Please input your answer in this cell:**